# Creating SQLite Database in Python

**SQLite is easy to use in Python, simply by importing the `sqlite3` module and connecting to a database.**

In [1]:
import sqlite3

In [2]:
db = sqlite3.connect('contacts/contacts.sqlite')

**SQLite does not care what you call the database, as long as it is a valid filename, i.e. you don't need `.db` extension.**

**In order to create a table and insert data, you need to pass each SQL statement in an `execute()` method, called off the database object - make sure to use double quotation marks to contain SQL statement. You do not use a semi-colon to terminate the SQL statements because Python automatically concatenates it for you.**

In [3]:
db.execute("CREATE TABLE IF NOT EXISTS contacts (name TEXT NOT NULL, phone INTEGER, email TEXT)")

db.execute("INSERT INTO contacts (name, phone, email) VALUES('Shely', 1234567, 'shely@email.com')")
db.execute("INSERT INTO contacts (name, phone, email) VALUES('Ryan', 2345678, 'ryan@email.com')")
db.execute("INSERT INTO contacts (name, phone, email) VALUES('Betty', 3456789, 'betty@email.com')")
db.execute("INSERT INTO contacts (name, phone, email) VALUES('Jacinta', 4567890, 'jacinta@email.com')")

**Now you can query the database, using a `cursor`, which is an *iterable*:**

In [4]:
cursor = db.cursor()

cursor.execute("SELECT * FROM contacts")

for name, phone, email in cursor:
    print(f"{name}: {phone} | {email}")

Shely: 1234567 | shely@email.com
Ryan: 2345678 | ryan@email.com
Betty: 3456789 | betty@email.com
Jacinta: 4567890 | jacinta@email.com


**Once you have iterated over a query result, the cursor object is 'emptied' and you need to create a new one to run another query. Note how after the `fetchall()` method is used, the cursor object has been emptied and does not print the `for` loop.**

In [5]:
cursor = db.cursor()

cursor.execute("SELECT * FROM contacts")

print(cursor.fetchall())

for name, phone, email in cursor:
    print(f"{name}: {phone} | {email}")

[('Shely', 1234567, 'shely@email.com'), ('Ryan', 2345678, 'ryan@email.com'), ('Betty', 3456789, 'betty@email.com'), ('Jacinta', 4567890, 'jacinta@email.com')]


In [6]:
cursor = db.cursor()

cursor.execute("SELECT * FROM contacts")

print(cursor.fetchone())
print(cursor.fetchone())
print(cursor.fetchone())
print(cursor.fetchone())
print(cursor.fetchone())

('Shely', 1234567, 'shely@email.com')
('Ryan', 2345678, 'ryan@email.com')
('Betty', 3456789, 'betty@email.com')
('Jacinta', 4567890, 'jacinta@email.com')
None


**Technically, you don't need to create the cursor to make an SQL connection, since you can execute an SQL statement off the database, which is automatically connected. You can think of it as a shortcut.**

In [7]:
for row in db.execute("SELECT * FROM contacts"):
    print(row)

('Shely', 1234567, 'shely@email.com')
('Ryan', 2345678, 'ryan@email.com')
('Betty', 3456789, 'betty@email.com')
('Jacinta', 4567890, 'jacinta@email.com')


**NOTE: After working on a database, you must close all the connections at the end (cursor and database).**

**If open the contacts database again after closing it, you will see that all `INSERTS` and `UPDATES` made before have been lost. This is because you did not 'commit' the changes (think GIT).**

In [9]:
cursor.close()

db.close()

In [10]:
db = sqlite3.connect('contacts/contacts.sqlite')

In [11]:
for row in db.execute("SELECT * FROM contacts"):
    print(row)

## Committing changes to database

**Any changes made to a connected database, i.e. `INSERT`, `UPDATE`, and `DELETE` statements, must be committed if you want to save the changes. You can modify a database as much as you want but the original stays intact until those SQL statements have been committed.**

In [12]:
db.execute("CREATE TABLE IF NOT EXISTS contacts (name TEXT NOT NULL, phone INTEGER, email TEXT)")

db.execute("INSERT INTO contacts (name, phone, email) VALUES('Shely', 1234567, 'shely@email.com')")
db.execute("INSERT INTO contacts (name, phone, email) VALUES('Ryan', 2345678, 'ryan@email.com')")
db.execute("INSERT INTO contacts (name, phone, email) VALUES('Betty', 3456789, 'betty@email.com')")
db.execute("INSERT INTO contacts (name, phone, email) VALUES('Jacinta', 4567890, 'jacinta@email.com')")

In [13]:
cursor = db.cursor()

cursor.execute("SELECT * FROM contacts")

print(cursor.fetchall())

[('Shely', 1234567, 'shely@email.com'), ('Ryan', 2345678, 'ryan@email.com'), ('Betty', 3456789, 'betty@email.com'), ('Jacinta', 4567890, 'jacinta@email.com')]


In [14]:
cursor.close()

# COMMIT CHANGES
db.commit()

db.close()

In [15]:
db = sqlite3.connect('contacts/contacts.sqlite')

for name, phone, email in db.execute("SELECT * FROM contacts"):
    print(f"{name}: {phone} | {email}")

Shely: 1234567 | shely@email.com
Ryan: 2345678 | ryan@email.com
Betty: 3456789 | betty@email.com
Jacinta: 4567890 | jacinta@email.com


In [18]:
# Update database through cursor connection (rather than database connection)

update_sql = "UPDATE contacts SET email='update@update.com' WHERE contacts.phone=1234567"

update_cursor = db.cursor()

update_cursor.execute(update_sql)

print(f"{update_cursor.rowcount} rows updated")

1 rows updated


In [19]:
for name, phone, email in db.execute("SELECT * FROM contacts"):
    print(f"{name}: {phone} | {email}")

Shely: 1234567 | update@update.com
Ryan: 2345678 | ryan@email.com
Betty: 3456789 | betty@email.com
Jacinta: 4567890 | jacinta@email.com


In [20]:
# COMMIT CHANGES VIA THE CURSOR OBJECT

update_cursor.connection.commit()

#print("Are connections the same? {}".format(update_cursor.connection == db))

update_cursor.close()

db.close()

**Committing changes made to the cursor object is most common when making many updates in a large script, i.e. you commit the changes as you go along, rather than committing all the changes on the database at the end (both commits do the same thing). As programs get more complex, this usefulness will become evident.**

**In practice, new updates or records would be provided as variables, rather han hard-coded into the SQL statement.**

    new_name = 'Barry'
    new_email = 'barry@email.com'
    new_phone = 5678901

    insert_sql = "INSERT INTO contacts (name, phone, email) VALUES({}, {}, {})".format(new_name, new_phone, new_email)

    db.execute(insert_sql)

    db.commit()
    
**You can include multiple SQL statements with `executescript()` method, where the individual statements are separated by semi-colons making sure to include a terminating semi-colon on the last statement. Double and triple quotation marks will suffice to contain the statements.**

In [4]:
db = sqlite3.connect('contacts/contacts.sqlite')

update_cursor = db.cursor()

update_sql = "UPDATE contacts SET email='new@email.com' WHERE contacts.phone=2345678; SELECT * FROM contacts;"

update_cursor.executescript(update_sql)

In [5]:
for name, phone, email in db.execute("SELECT * FROM contacts"):
    print(f"{name}: {phone} | {email}")

Shely: 1234567 | update@update.com
Ryan: 2345678 | new@email.com
Betty: 3456789 | betty@email.com
Jacinta: 4567890 | jacinta@email.com


In [6]:
update_cursor.connection.commit()

update_cursor.close()

In [7]:
new_table_sql = """CREATE TABLE IF NOT EXISTS jobs (_id INTEGER NOT NULL, name TEXT PRIMARY KEY, role TEXT NOT NULL);
INSERT INTO jobs (_id, name, role) VALUES(1, 'Shely', 'Manager');
INSERT INTO jobs (_id, name, role) VALUES(2, 'Ryan', 'Deputy Manager');
INSERT INTO jobs (_id, name, role) VALUES(3, 'Betty', 'Shop Assistant');
INSERT INTO jobs (_id, name, role) VALUES(4, 'Jacinta', 'Shop Assistant');"""

db.executescript(new_table_sql)

for _id, name, role in db.execute("SELECT * FROM jobs"):
    print(f"{name}: {role}")

Shely: Manager
Ryan: Deputy Manager
Betty: Shop Assistant
Jacinta: Shop Assistant


In [8]:
db.commit()

db.close()